## This notebook contains steps for configuring an instance of SQL Server running in a docker container.  It uses PowerShell functions in the DBATools module.  The following script will check if DBATools is installed.  If it is not, it will install it.  If it is installed, it will be updated to the latest version.

In [3]:
$exists = (Get-InstalledModule -Name DBATools).Name;
$exists;

if(!$exists)
{
    Install-Module -Name DBATools;   
}
else
{
    Update-DbaTools;
}

## Once DBATools is installed, you are ready to start.  The first step sets max server memory (mb) to a recommended value.

In [13]:
$instance = 'localhost,1436';
$path = 'C:\Users\fgill\Documents\GitHub\sqlconfig';
$username = 'sa';
$password = 'JuanS0t022';
Set-Location -Path $path;

Invoke-Sqlcmd -ServerInstance $instance -Database master -InputFile '.\CreateSqlMaxMemoryProc.sql' -UserName $username -Password $password;

Invoke-Sqlcmd -ServerInstance $instance -Database master -Query 'EXEC SetSqlServerMaxMemory @noexec = 0;' -UserName $username -Password $password;


Total Physical Memory Total OS Reserve Expected SQL Server Memory Current Configured Value
--------------------- ---------------- -------------------------- ------------------------
                    3                1 2048                                           2048




## The next step downloads and installs Adam Machanic's sp_whoisactive stored procedure.

In [18]:
$DBAToolsData = Get-DbatoolsConfigValue -FullName "Path.DbatoolsData";
$instance = 'localhost,1436';
$temp = ([System.IO.Path]::GetTempPath())
$zipfile = Join-Path -Path $temp -ChildPath "spwhoisactive.zip";

$baseUrl = "https://github.com/amachanic/sp_whoisactive/archive"
$latest = (((Invoke-WebRequest -UseBasicParsing -uri https://github.com/amachanic/sp_whoisactive/releases/latest).Links | Where-Object { $PSItem.href -match "zip" } | Select-Object href -First 1).href -split '/')[-1]

$url = $baseUrl + "/" + $latest
    
$LocalCachedCopy = Join-Path -Path $DbatoolsData -ChildPath $latest;

Invoke-WebRequest $url -OutFile $zipfile -ErrorAction Stop -UseBasicParsing
Copy-Item -Path $zipfile -Destination $LocalCachedCopy

Unblock-File $zipfile -ErrorAction SilentlyContinue
Expand-Archive -Path $zipfile -DestinationPath $temp -Force

$sqlfile = (Get-ChildItem "$($temp)who*active*.sql" -Recurse -ErrorAction SilentlyContinue | Select-Object -First 1).FullName

Invoke-Sqlcmd -ServerInstance $instance -Database master -InputFile $sqlfile -Username sa -Password JuanS0t022;

Remove-Item -Path $zipfile;
Remove-Item -Path $sqlfile;